# Geographic Visualization

### 1. Import Libraries and Datasets
### 2. Data Wrangling
### 3. Choropleth Maps
### 4. Save Data

## Import Libraries and Datasets

Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import os
import folium
import json

In [2]:
# This propts matplotlib visuals to appear in the notebook 
%matplotlib inline

Import Data

In [3]:
# path shortcut
path = r'C:\Users\kacee\OneDrive\Desktop\Project6'

In [4]:
# Importing Merged data
df_m = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'Merged_3_Data.csv'), index_col = False)

In [5]:
# Importing Geodata for US States
us_data = r'C:\Users\kacee\OneDrive\Desktop\Project6\02 Data\Original Data\us_states.json'

In [6]:
f = open(r'C:\Users\kacee\OneDrive\Desktop\Project6\02 Data\Original Data\us_states.json', encoding='utf-8')
# returns JSON object asa dictionary
data = json.load(f)
# Iterating through the json list
for i in data['features']:
    print(i)

{'type': 'Feature', 'id': 'AL', 'properties': {'name': 'Alabama'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-87.359296, 35.00118], [-85.606675, 34.984749], [-85.431413, 34.124869], [-85.184951, 32.859696], [-85.069935, 32.580372], [-84.960397, 32.421541], [-85.004212, 32.322956], [-84.889196, 32.262709], [-85.058981, 32.13674], [-85.053504, 32.01077], [-85.141136, 31.840985], [-85.042551, 31.539753], [-85.113751, 31.27686], [-85.004212, 31.003013], [-85.497137, 30.997536], [-87.600282, 30.997536], [-87.633143, 30.86609], [-87.408589, 30.674397], [-87.446927, 30.510088], [-87.37025, 30.427934], [-87.518128, 30.280057], [-87.655051, 30.247195], [-87.90699, 30.411504], [-87.934375, 30.657966], [-88.011052, 30.685351], [-88.10416, 30.499135], [-88.137022, 30.318396], [-88.394438, 30.367688], [-88.471115, 31.895754], [-88.241084, 33.796253], [-88.098683, 34.891641], [-88.202745, 34.995703], [-87.359296, 35.00118]]]}}
{'type': 'Feature', 'id': 'AK', 'properties': {'name': 'Alaska'},

## Data Wrangling

In [7]:
df_m.head()

,Unnamed: 0,Year,State,House Cost,Income,State_Minimum_Wage,House Category,Income Category
0,0,2008,AL,137250.036361,44480.0,5.85,Low Median Price,Low Income
1,1,2008,AR,114877.181812,39590.0,6.25,Low Median Price,Low Income
2,2,2008,AZ,201922.818171,46910.0,6.90,Middle Median Price,Low Income
3,3,2008,CA,336043.181811,57010.0,8.00,High Median Price,Middle Income
4,4,2008,CO,230399.352287,60940.0,7.02,Middle Median Price,Middle Income


In [8]:
df_m.shape

(720, 8)

In [9]:
df_m = df_m.drop(columns =['Unnamed: 0'])

In [10]:
df_m.head()

,Year,State,House Cost,Income,State_Minimum_Wage,House Category,Income Category
0,2008,AL,137250.036361,44480.0,5.85,Low Median Price,Low Income
1,2008,AR,114877.181812,39590.0,6.25,Low Median Price,Low Income
2,2008,AZ,201922.818171,46910.0,6.90,Middle Median Price,Low Income
3,2008,CA,336043.181811,57010.0,8.00,High Median Price,Middle Income
4,2008,CO,230399.352287,60940.0,7.02,Middle Median Price,Middle Income


In [11]:
df_m.shape

(720, 7)

In [12]:
df_m.columns

Index(['Year', 'State', 'House Cost', 'Income', 'State_Minimum_Wage',
       'House Category', 'Income Category'],
      dtype='object')

Create new columns for maps.
1. Multiplied_Income = House cost / Income = Shows the cost of the averge house and how many times it is more than the averge income
2. Income_Percentage = Income / House cost x 100 = Shows What percent the averge income is compared to the cost of the averge house

In [13]:
# Calculating the percentage
df_m['Multiplied_Income'] = (df_m['House Cost'] / df_m['Income'])

In [14]:
# Calculating the percentage
df_m['Multiplied_Wage'] = (df_m['House Cost'] / (df_m['State_Minimum_Wage']*2080))

In [15]:
# Calculating the percentage
df_m['Income_Percentage'] = (df_m['Income'] / df_m['House Cost']) * 100

In [16]:
df_m['Minimum_Wage_Yearly'] = (df_m['State_Minimum_Wage']*2080)

In [17]:
df_m['Income_Percentage'].replace(np.inf, np.nan, inplace=True)

In [20]:
df_m.tail(50)

,Year,State,House Cost,Income,State_Minimum_Wage,House Category,Income Category,Multiplied_Income,Multiplied_Wage,Income_Percentage,Minimum_Wage_Yearly
670,2021,WV,151150.000000,46840.0,7.25,Middle Median Price,Low Income,3.226943,10.023210,30.989084,15080.0
671,2021,WY,264427.777800,71050.0,7.25,Middle Median Price,Middle Income,3.721714,17.534999,26.869341,15080.0
672,2022,AL,216632.541667,59910.0,7.25,Middle Median Price,Middle Income,3.615966,14.365553,27.655125,15080.0
673,2022,AR,201002.222211,53980.0,11.00,Middle Median Price,Low Income,3.723643,8.785062,26.855424,22880.0
674,2022,AZ,384259.287044,73450.0,12.80,High Median Price,Middle Income,5.231576,14.432816,19.114697,26624.0
675,2022,CA,608345.677783,85300.0,15.00,High Median Price,High Income,7.131837,19.498259,14.021633,31200.0
676,2022,CO,569348.636914,89930.0,12.56,High Median Price,High Income,6.331020,21.793416,15.795243,26124.8
677,2022,CT,341104.166660,90730.0,14.00,High Median Price,High Income,3.759552,11.713742,26.598913,29120.0
678,2022,DE,325295.984850,80750.0,10.50,High Median Price,High Income,4.028433,14.894505,24.823546,21840.0
679,2022,FL,349895.703332,65370.0,11.00,High Median Price,Middle Income,5.352543,15.292644,18.682710,22880.0


## Choropleth Maps

Make Subsests for each year

In [19]:
df08 = df_m[df_m['Year'] == 2008]
df08

,Year,State,House Cost,Income,State_Minimum_Wage,House Category,Income Category,Multiplied_Income,Multiplied_Wage,Income_Percentage,Minimum_Wage_Yearly
0,2008,AL,137250.036361,44480.0,5.85,Low Median Price,Low Income,3.085657,11.279589,32.408006,12168.0
1,2008,AR,114877.181812,39590.0,6.25,Low Median Price,Low Income,2.901672,8.836706,34.462893,13000.0
2,2008,AZ,201922.818171,46910.0,6.90,Middle Median Price,Low Income,4.304473,14.069316,23.231649,14352.0
3,2008,CA,336043.181811,57010.0,8.00,High Median Price,Middle Income,5.894460,20.194903,16.965082,16640.0
4,2008,CO,230399.352287,60940.0,7.02,Middle Median Price,Middle Income,3.780757,15.779048,26.449727,14601.6
5,2008,CT,269690.000000,64680.0,7.65,Middle Median Price,Middle Income,4.169604,16.948844,23.983092,15912.0
6,2008,DE,210382.954550,50700.0,7.15,Middle Median Price,Low Income,4.149565,14.146245,24.098911,14872.0
7,2008,FL,191100.059283,44860.0,6.79,Middle Median Price,Low Income,4.259921,13.530932,23.474613,14123.2
8,2008,GA,135955.603904,46230.0,5.15,Low Median Price,Low Income,2.940852,12.691897,34.003747,10712.0
9,2008,HI,395057.272733,61520.0,7.25,High Median Price,Middle Income,6.421607,26.197432,15.572426,15080.0


In [20]:
df09 = df_m[df_m['Year'] == 2009]

In [21]:
df10 = df_m[df_m['Year'] == 2010]

In [22]:
df11 = df_m[df_m['Year'] == 2011]

In [23]:
df12 = df_m[df_m['Year'] == 2012]

In [24]:
df13 = df_m[df_m['Year'] == 2013]

In [25]:
df14 = df_m[df_m['Year'] == 2014]

In [26]:
df15 = df_m[df_m['Year'] == 2015]

In [27]:
df16 = df_m[df_m['Year'] == 2016]

In [28]:
df17 = df_m[df_m['Year'] == 2017]

In [29]:
df18 = df_m[df_m['Year'] == 2018]

In [30]:
df19 = df_m[df_m['Year'] == 2019]

In [31]:
df20 = df_m[df_m['Year'] == 2020]

In [32]:
df21 = df_m[df_m['Year'] == 2021]

In [33]:
df22 = df_m[df_m['Year'] == 2022]

In [34]:
map1 = folium.Map(location = [40, -95], zoom_start = 4)

folium.Choropleth(
    geo_data = us_data, 
    data = df08,
    columns = ['State', 'House Cost'],
    key_on = 'id',
    fill_color = 'YlOrBr', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Housing Cost 08").add_to(map1)

folium.LayerControl().add_to(map1)

map1

In [35]:
map2 = folium.Map(location = [40, -95], zoom_start = 4)

folium.Choropleth(
    geo_data = us_data, 
    data = df22,
    columns = ['State', 'House Cost'],
    key_on = 'id',
    fill_color = 'YlOrBr', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Housing Cost 2022").add_to(map2)

folium.LayerControl().add_to(map2)

map2

In [36]:
map3 = folium.Map(location = [40, -95], zoom_start = 4)

folium.Choropleth(
    geo_data = us_data, 
    data = df08,
    columns = ['State', 'Multiplied_Income'],
    key_on = 'id',
    fill_color = 'YlOrBr', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "House Cost / Income 2008").add_to(map3)

folium.LayerControl().add_to(map3)

map3

In [37]:
map4 = folium.Map(location = [40, -95], zoom_start = 4)

folium.Choropleth(
    geo_data = us_data, 
    data = df22,
    columns = ['State', 'Multiplied_Income'],
    key_on = 'id',
    fill_color = 'YlOrBr', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "House Cost / Income 2022").add_to(map4)

folium.LayerControl().add_to(map4)

map4

In [38]:
map5 = folium.Map(location = [40, -95], zoom_start = 4)

folium.Choropleth(
    geo_data = us_data, 
    data = df08,
    columns = ['State', 'Income_Percentage'],
    key_on = 'id',
    fill_color = 'YlOrBr', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Averge Income % for Averge House Cost 2008").add_to(map5)

folium.LayerControl().add_to(map5)

map5

In [39]:
map6 = folium.Map(location = [40, -95], zoom_start = 4)

folium.Choropleth(
    geo_data = us_data, 
    data = df22,
    columns = ['State', 'Income_Percentage'],
    key_on = 'id',
    fill_color = 'YlOrBr', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Averge Income % for Averge House Cost 2022").add_to(map6)

folium.LayerControl().add_to(map6)

map6

## Save Data

In [40]:
map1.save('Housing_Cost_Map_2008.html')

In [41]:
map2.save('Housing_Cost_Map_2022.html')

In [42]:
map3.save('Multiplied_Income_Map_2008.html')

In [43]:
map4.save('Multiplied_Income_Map_2022.html')

In [44]:
map5.save('Income_Percentage_Map_2008.html')

In [45]:
map6.save('Income_Percentage_Map_2022.html')

In [35]:
# Export Merged/Modified Dataset
df_m.to_csv(os.path.join(path, '02 Data','Prepared Data', 'Geospatial_Complete_Data.csv'))

## Does the analysis answer any of your existing research questions?
## Does the analysis lead you to any new research questions?

1. The analysis shows in different ways how income relates to housing cost across the states from 2008-2022. 
2. New question: Do missing values affect the overall outcome of these maps?
3. New question: Is there a way to make a more comprehensive map?